In [1]:
import pandas as pd
import numpy as np
import math
import collections
import re
import Queue
import copy

In [2]:
class Floor:
    def __init__(self, floorid, microchips, generators):
        self.floor_id = floorid
        self.microchips = microchips
        self.generators = generators
    
    def  compare(self,other_floor):
        if len(self.generators) != len(other_floor.generators):
            return False

        if len(self.microchips) != len(other_floor.microchips):
            return False
        
        if len(self.generators) == 0 or len(self.microchips) == 0:
            return True
        
        for g0 in self.generators:
            if g0 not in other_floor.generators:
                return False
            
        for g1 in other_floor.generators:
            if g1 not in self.generators:
                return False

        for m0 in self.microchips:
            if m0 not in other_floor.microchips:
                return False
            
        for m1 in other_floor.microchips:
            if m1 not in self.microchips:
                return False
            
        return True
    
    def connected_RTGs(self):
        connected = 0
        for generator in self.generators:
            if generator in self.microchips:
                connected += 1
        return connected
    

In [3]:
#### branches
#### enumerate through possible valid 
#### combinations and floor transfers for the elevator
class Building():
    def __hash__(self):
        hash(self.floors) 
    def __init__(self,elevator='first',floors={},depth=0):
        self.floors = floors
        self.elevator = elevator
        self.depth = depth
 
    def progress(self):
        return -self.floors['fourth'].connected_RTGs()
    
    def __eq__(self,other_building):
        if self.elevator != other_building.elevator:
            return False

        if self.floors['fourth'].compare(other_building.floors['fourth']) == False:
            return False
        
        floors_matched = []
        for floor_key,floor in self.floors.iteritems():
            floor_matched = False
            if floor_key == 'fourth':
                continue
            for ofloor_key,ofloor in other_building.floors.iteritems():
                if ofloor_key == 'fourth':
                    continue
                if ofloor_key not in floors_matched and floor.compare(ofloor) == True:
                    floor_matched = True
                    floors_matched.append(ofloor_key)
                    break
            if floor_matched == False:
                return False
        return True
    
    def construct(self,instructions):
        #print lines
        for line in instructions:
            floor_id = line.split(" ")[1]

            ## add all of the generators
            generator_list = re.findall(r' (\w+\s+generator)',line)
            generators = {}
            for generator in generator_list:
                generators[generator.split(" ")[0][0:2]] = True

            ## add all of the microchips
            microchips = {}
            microchip_list = re.findall(r' (\w+-\w+\s+microchip)',line)
            for microchip in microchip_list:
                microchips[microchip.split("-")[0][0:2]] = True

            floor = Floor(floor_id, microchips, generators)
            self.floors[floor.floor_id] = floor 
            
    def spawnValidMoves(self):
        validcombos = []
        
        currentfloor = self.floors[self.elevator]
        
        ## enumerate through generator/microchip combinations

        
        ########################### PAIR MOVE ##################################################
        ## figure out the valid connected RTGs that can be moved from the floor
        valid_CRTGS = []
        for g in currentfloor.generators:
            for m in currentfloor.microchips:
                if g == m:
                    valid_CRTGS.append((g,m))
                if len(valid_CRTGS) > 0:
                    break
            if len(valid_CRTGS) > 0:
                break
                    
        ## figure out the valid generator pairs that can be moved from the floor
        valid_generators = []
        idx0 = 0
        for g0 in currentfloor.generators:
            idx1 = 0
            for g1 in currentfloor.generators:
                ## ok this is a generator pair, but will it melt anything
                ## if it leaves this floor
                if idx1 > idx0:
                    ## there are no microchips here
                    if len(currentfloor.microchips) == 0:
                        valid_generators.append((g0,g1))
                    else:
                        ## oh man
                        ## case 0: there are no more generators left
                        if len(currentfloor.generators) == 2:
                            valid_generators.append((g0,g1))
                        else:
                            safe = True
                            ## are we disconnecting some chips??
                            if g0 in currentfloor.microchips or g1 in currentfloor.microchips:
                                ## make there are not any disconnected gnerators that will fry these puppies
                                for g2 in currentfloor.generators:
                                    if g2 != g0 and g2 != g1 and g2 not in currentfloor.microchips:
                                        safe = False
                            if safe == True:
                                valid_generators.append((g0,g1))
                              
                        #cRTGs = currentfloor.connectedRTGs()
                        
                        
                idx1 += 1
            idx0 += 1
        
        ## figure out the valid microchip pairs that can be moved from the floor
        valid_microchips = []
        idx0 = 0
        for m0 in currentfloor.microchips:
            idx1 = 0
            for m1 in currentfloor.microchips:
                ## Ok this is a microchip pair, but will it melt anything if it leaves this floor
                if idx1 > idx0:
                    ## there are no generators to disconnect
                    if len(currentfloor.generators) == 0:
                        valid_microchips.append((m0,m1))
                    else:
                        ## these are the only microhcips on the floor
                        if len(currentfloor.microchips) == 2:
                            valid_microchips.append((m0,m1))
                        else:
                            safe = True
                            ## are we disconnecting some chips??
                            if m0 in currentfloor.generators or m1 in currentfloor.generators:
                                for m2 in currentfloor.microchips:
                                    if m2  != m0 and m2 != m1 and m2 not in currentfloor.generators:
                                        safe = False
                            if safe == True:
                                valid_microchips.append((m0,m1))
                                  
                idx1 += 1
            idx0 += 1
        ############################# PAIR MOVE #####################################################
        
        ############################# SOLO MOVE #####################################################
        for g in currentfloor.generators:
            ## there are no microchips to melt
            if len(currentfloor.microchips) == 0:
                valid_generators.append((g,"empty"))
            ## this generator is connected to a microchip
            elif g in currentfloor.microchips:
                safe = True
                for g1 in currentfloor.generators:
                    if g1 != g and g1 not in currentfloor.microchips:
                        safe = False
                if safe == True:
                    valid_generators.append((g,"empty"))
            else:
                valid_generators.append((g,"empty"))
           
        
        for m in currentfloor.microchips:
            ## there are no generators to melt microchips
            if len(currentfloor.generators) == 0:
                valid_microchips.append((m,"empty"))
            ## this microchip is connected to a generator
            elif m in currentfloor.generators:
                safe = True
                for m1 in currentfloor.microchips:
                    if m1 != m and m1 not in currentfloor.generators:
                        safe = False
                if safe == True:
                    valid_microchips.append((m,"empty"))
            else:
                valid_microchips.append((m,"empty"))
        ####################################### FINISHED BUILDING THE VALID potential moves ##############        

        
        
        validmoves = []
        
        ## iterate through possible elevator moves
        for floor_name,floor in self.floors.iteritems():
            ## elevator is here already
            if floor_name == self.elevator:
                continue
            else:
                for RTG in valid_CRTGS:
                    ## easy a connected RTG can go anywhere
                    ## construct new building
                    ## 1. copy the floors
                    new_floors = copy.deepcopy(self.floors)

                    ## remove the microchip pair from the current floor
                    new_floors[self.elevator].generators.pop(RTG[0])
                    new_floors[self.elevator].microchips.pop(RTG[0])
                    new_floors[floor_name].generators[RTG[0]] = True
                    new_floors[floor_name].microchips[RTG[0]] = True

                    validmoves.append(Building(floor_name,new_floors,self.depth+1))
                
                ## these are disconnected generators, they cannot move to floors that contain
                ## disconnected microchips not connected by moving these generators to the floor
                for gpair in valid_generators:
                    safe = True
                    for m in floor.microchips:
                        ## this will not be connected by the microchip
                        if m not in gpair and m not in floor.generators:
                            safe = False
                        
                    if safe == True:
                        ## construct new building
                        ## 1. copy the floors
                        new_floors = copy.deepcopy(self.floors)
                        
                        ## remove the microchip pair from the current floor
                        new_floors[self.elevator].generators.pop(gpair[0])
                        new_floors[floor_name].generators[gpair[0]] = True
                    
                        ## move them to the new floor
                        if gpair[1] != "empty":
                            new_floors[self.elevator].generators.pop(gpair[1])
                            new_floors[floor_name].generators[gpair[1]] = True
                            
                        validmoves.append(Building(floor_name,new_floors,self.depth+1))
                            
                            
                ## these are disconnected microchips, they cannot move to floors that contain disconnected
                ## generators not connected by moving these microchips to the floor
                for mpair in valid_microchips:
                    safe = True
                    for g in floor.generators:
                        if g not in mpair and g not in floor.microchips:
                            safe = False
                    
                    if safe == True:
                        ## construct new building
                        ## 1. copy the floors
                        new_floors = copy.deepcopy(self.floors)
                        ## remove the microchip pair from the current floor
                        new_floors[self.elevator].microchips.pop(mpair[0])
                        new_floors[floor_name].microchips[mpair[0]] = True
                    
                        ## move them to the new floor
                        if mpair[1] != "empty":
                            new_floors[self.elevator].microchips.pop(mpair[1])
                            new_floors[floor_name].microchips[mpair[1]] = True
                            
                        validmoves.append(Building(floor_name,new_floors,self.depth+1))
        return validmoves

In [4]:
from heapq import *
import itertools

pq = []                         # list of entries arranged in a heap
entry_finder = {}               # mapping of tasks to entries
REMOVED = '<removed-task>'      # placeholder for a removed task
counter = itertools.count()     # unique sequence count

def add_task(task, priority=0):
    'Add a new task or update the priority of an existing task'
    if task in entry_finder:
        remove_task(task)
    count = next(counter)
    entry = [priority, count, task]
    entry_finder[task] = entry
    heappush(pq, entry)

def remove_task(task):
    'Mark an existing task as REMOVED.  Raise KeyError if not found.'
    entry = entry_finder.pop(task)
    entry[-1] = REMOVED

def pop_task():
    'Remove and return the lowest priority task. Raise KeyError if empty.'
    while pq:
        priority, count, task = heappop(pq)
        if task is not REMOVED:
            del entry_finder[task]
            return task
    raise KeyError('pop from an empty priority queue')
    

In [5]:
data = pd.read_csv('./input.txt',header=None,delimiter="\n")

instructions = data[0].values

root = Building()
root.construct(instructions)
print root.floors['first'].generators

add_task(root,root.progress())

visitedlist = []
visitlist.append(root)
print visitlist
currentnode = root


## how many RTGs do we need connected?
goal = 0
for floor_name,floor in root.floors.iteritems():
    for g in floor.generators:
        goal += 1
## nodes to expand
max_depth = 50
finished = False
depth = 0
best_state = 0
current_idx = 0
while finished == False and depth < max_depth:
    if current_idx % 1000 == 0:
        print current_idx
    ## grab the next node
    currentnode = pop_task()
    
    ## check to see if its the goal
    
    ## add children to the visit list
    valid_moves = currentnode.spawnValidMoves()
    for move in valid_moves:
        if move not in visitedlist:
            add_task(move,move.progress())
            
    
    finished_RTGs = currentnode.floors['fourth'].connected_RTGs() 
    if finished_RTGs > best_state:
        best_state = finished_RTGs
        print "youre the best around: " + str(best_state) + currentnode.depth
        
    if best_state == goal:
        finished = True
        
    visitedlist.append(currentnode)
    current_idx += 1

{'pr': True, 'ru': True, 'co': True, 'po': True, 'th': True}


TypeError: unhashable instance